In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D

import numpy as np
import subprocess
import os
import copy

import netCDF4 as nc
import csv
import toml
from tomlkit import dumps
from tomlkit import parse
from tomlkit import inline_table

#import yaml
from ruamel.yaml import YAML
yaml = YAML()
yaml.preserve_quotes = True  # Keep quotes if they exist
yaml.indent(mapping=2, sequence=4, offset=2)  # Preserve indentation
yaml.representer.add_representer(
    float, lambda dumper, data: dumper.represent_scalar("tag:yaml.org,2002:float", str(data))
)

with open("../config/masterconfig.yaml", "r") as file:
    config = yaml.load(file) or {}
AGNI_rootpath = config['AGNI_rootpath']
AGNI_outputpath = config['AGNI_outputpath']
SOCRATES_path = config['SOCRATES_path']
REPO_path = config['REPO_path']# the path of the LIFEredoxsurvey repo
fastchem_path = config['fastchem_rootpath']

In [ ]:
filepath = "../output/atmospheres_mole.csv"
mole_data = np.genfromtxt(filepath,skip_header=1,delimiter=',')[:,2:]
coords = np.genfromtxt(filepath,skip_header=1,delimiter=',')[:,:2]
titles = np.genfromtxt(filepath,dtype=str,delimiter=',')[0][2:]

In [ ]:
def input_volatiles(filepath,volatile_dict):

    # open toml file
    with open(filepath, "r") as f:
        toml_str = f.read()
    doc = parse(toml_str)

    # input volatile dictionary
    my_table = inline_table()
    for key, value in volatile_dict.items():
        my_table[key] = value
    my_table.trivia.trail = "\n"
    doc["composition"]["vmr_dict"] = my_table

    # write to toml file
    with open(filepath, "w") as f:
        f.write(dumps(doc))  # Use str() to serialize back to TOML

def write_data(filepath,bandcenter,intensity):
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['bandcenter','intensity',])
        for i in range(len(bandcenter)):
            string = [str(bandcenter[i]),str(intensity[i])]
            writer.writerow(string)

In [ ]:
for i in range(len(mole_data)):

    # insert atmosphere and run AGNI
    dictionary = dict(zip(titles, mole_data[i]))
    input_volatiles("../config/LIFE_spectra.toml",dictionary)
    prompt = "export FC_DIR=" + fastchem_path + ";export RAD_DIR=" + SOCRATES_path + ";./agni.jl " + REPO_path + "/config/LIFE_spectra.toml"
    subprocess.run(prompt,cwd=AGNI_rootpath, shell=True)

    # read out emission data
    AGNI_out = AGNI_outputpath + "/atm.nc"
    ds = nc.Dataset(AGNI_out)
    bandmin_data = ds['bandmin'][:]
    bandmax_data = ds['bandmax'][:]
    bandcenter = (bandmin_data + bandmax_data)/2 * 10**9
    bandwidth = (bandmax_data - bandmin_data) * 10**9
    lw_data = ds['ba_U_LW'][:]
    sw_data = ds['ba_U_SW'][:]
    bandflux = (lw_data[1, :] + sw_data[1, :]) / bandwidth * 1000

    # check if the same species appear in every run
    AGNI_gases = []
    for j in range(len(ds['gases'])):
        molecule_name = []
        for k in range(10):
            if ds['gases'][:][j][k].decode('UTF-8') == ' ':
                break
            molecule_name.append(ds['gases'][:][j][k].decode('UTF-8'))
        AGNI_gases.append(''.join(molecule_name))
    if i == 0:
        gas_check = copy.deepcopy(AGNI_gases)
        with open("../output/AGNI_atmospherecomposition", 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['fo2', 'C/H'] + gas_check)
    if AGNI_gases != gas_check:
            with open("../output/AGNI/error",'w',newline='') as f:
                writer = csv.writer(f)
                writer.writerow([i,str(round(coords[i][0],2)),str(round(coords[i][1],2))])

    # read out composition data
    x_gas = ds['x_gas'][:][-1]
    with open("../output/AGNI_atmospherecomposition",'a',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(list([round(coords[i][0],2),round(coords[i][1],2)]) + list(x_gas))

    # save text file
    csvname = '../output/AGNI/AGNI_textfiles/AGNI_spectrum_' + str(round(coords[i][0],2)) + '_' + str(round(coords[i][1],2)) + '.csv'
    write_data(csvname, bandcenter,bandflux)

    # save raw data
    foldername = '/dataserver/users/formingworlds/borgmann/output/AGNI/raw/AGNI_spectrum_' + str(round(coords[i][0],2)) + '_' + str(round(coords[i][1],2))
    try:
        subprocess.run("mkdir " + foldername, shell=True)
    finally:
        subprocess.run("cp -a " + AGNI_rootpath + "/out/. " + foldername + "/", shell=True)

In [ ]:
# find which molecules appear at concentration higher than 0.1%

file="../output/AGNI_atmospherecomposition"
data = np.genfromtxt(file,skip_header=1,delimiter=",")
datalength = len(data)
titles = np.genfromtxt(file,delimiter=",",dtype='str',skip_footer=datalength)

relevant_molecules = set()
relevant_molecules_index = set()

for i in range(len(data)):
    print(data[i][0],data[i][1],data[i][18])
    for j in range(2,len(data[i])):
        if data[i][j] > 0.001:
            relevant_molecules.add(titles[j])
            relevant_molecules_index.add(j)


print(relevant_molecules)

In [ ]:
# delete species that occur at >0.1%
cutlist = np.arange(51)
for i in list(relevant_molecules_index)[::-1]:
    cutlist = np.delete(cutlist,i,0)

data_cut = np.array(data)
titles_cut = np.array(titles)
for i in cutlist[::-1]:
    data_cut = np.delete(data_cut,i,1)
    titles_cut = np.delete(titles_cut,i,0)

In [ ]:
# pie charts for AGNI data

# order of volatiles is same as in EVo. Colours are assigned by me. 
#volatile_labels = ['N2' 'CO' 'H2' 'CH4' 'H2S' 'S2' 'SO2' 'H2O' 'CO2' 'O2' 'H2SO4' 'SO' 'NH3' 'OCS' 'S8']
#                  ['N2'      'H2' 'CH4' 'H2S'      'SO2' 'H2O' 'CO2'                   'NH3' '      S8']     
volatile_labels = titles_cut
volatile_colours = ["gray","magenta","red","green","pink","lime","orange","blue","cyan","lavender","brown","maroon","purple","navy","yellow"]

def savefigures(data,filename,plot_title):
   mpl.rcParams['figure.dpi'] = 800
   rasterplot = figure(figsize=(8,6))

   # setup custom order for rasterplot, since the subplots go left->right, up->down
   # but we need to go down->up, left->right
   custom_order = [9,5,1,10,6,2,11,7,3,12,8,4]

   # do plots
   for i in range(len(data)):
      frame = rasterplot.add_subplot(3,4,custom_order[i])
      frame.pie(data[i], labels=None,
         colors=volatile_colours)

   # add axes to the plot (I feel like I over-engineered this)
   master_ax = rasterplot.add_axes([0.1, 0.1, 0.8, 0.8])
   master_ax.patch.set_alpha(0)
   master_ax.set_zorder(-10)

   master_ax.spines['top'].set_visible(False)
   master_ax.spines['right'].set_visible(False)
   master_ax.spines['left'].set_linewidth(1.5)
   master_ax.spines['bottom'].set_linewidth(1.5)

   y_centers = [0.83, 0.50, 0.17] 
   master_ax.set_ylim(0, 1)
   master_ax.set_yticks(y_centers)
   master_ax.set_yticklabels(["5", "1", "0.1"])

   x_centers = [0.135, 0.365, 0.635, 0.865]
   master_ax.set_xlim(0, 1)
   master_ax.set_xticks(x_centers)
   master_ax.set_xticklabels(["-4", "-1.3", "1.3", "4"])

   master_ax.tick_params(labelsize=12)

   # add legend
   custom_lines = []
   for i in volatile_colours:
      custom_lines.append(Line2D([0], [0], color=i, lw=4))

   rasterplot.legend(custom_lines,volatile_labels,bbox_to_anchor=(1.05, 0.9))

   # axes labels
   rasterplot.supxlabel('Mantle $f$O$_2$, $\Delta$ IW')
   rasterplot.supylabel('Mantle C/H mass ratio')
   rasterplot.suptitle(plot_title)

   filepath = "../output/figures/" + filename
   plt.savefig(filepath)

savefigures(data_cut,'AGNI_3dplot_molefraction.png',"Molecular contents of volcanic gas parcel (mole fractions)")

In [ ]:
# raster plot of AGNI spectra

data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]

mpl.rcParams['figure.dpi'] = 800
rasterplot = figure(figsize=(8,6))

# setup custom order for rasterplot, since the subplots go left->right, up->down
# but we need to go down->up, left->right
custom_order = [9,5,1,10,6,2,11,7,3,12,8,4]

# do plots
for i in range(len(files)):
   frame = rasterplot.add_subplot(3,4,custom_order[i])
   image_path = data_path + "/" + files[i]
   bandcenter = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,0]
   bandflux = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,1]
   frame.plot(bandcenter/1000,bandflux,linewidth=0.5)
   frame.set_yscale('log')
   frame.set_xlim(4,18.5)
   frame.set_ylim(0.05,40)
   frame.set_yticklabels([])
   frame.set_xticklabels([])
   frame.set_xticks([])
   frame.set_yticks([])
   frame.minorticks_off()

# add axes to the plot (I feel like I over-engineered this)
master_ax = rasterplot.add_axes([0.1, 0.1, 0.8, 0.8])
master_ax.patch.set_alpha(0)
master_ax.set_zorder(-10)

master_ax.spines['top'].set_visible(False)
master_ax.spines['right'].set_visible(False)
master_ax.spines['left'].set_linewidth(1.5)
master_ax.spines['bottom'].set_linewidth(1.5)

y_centers = [0.83, 0.50, 0.17] 
master_ax.set_ylim(0, 1)
master_ax.set_yticks(y_centers)
master_ax.set_yticklabels(["5", "1", "0.1"])

x_centers = [0.135, 0.365, 0.635, 0.865]
master_ax.set_xlim(0, 1)
master_ax.set_xticks(x_centers)
master_ax.set_xticklabels(["-4", "-1.3", "1.3", "4"])

master_ax.tick_params(labelsize=12)


rasterplot.supxlabel('Mantle $f$O$_2$, $\Delta$ IW')
rasterplot.supylabel('Mantle C/H mass ratio')
rasterplot.suptitle("AGNI spectra for varying mantle $f$O$_2$ and mantle H/C mass ratio")
rasterplot.show()
rasterplot.savefig('../output/figures/AGNI_rasterplot.png')

In [ ]:
# save individual images
plt.ioff() # supress notebook output

data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]

mpl.rcParams['figure.dpi'] = 300

for i in range(len(files)):
#for i in [90]:
    image_path = data_path + "/" + files[i]
    bandcenter = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,0]
    bandflux = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,1]

    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bandcenter/1000,bandflux)
    frame1.set_xlabel('wavelength (microns)')
    frame1.set_ylabel('Spectral flux density (erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$)')
    #frame1.set_xscale('log')
    frame1.set_yscale('log')
    frame1.set_xlim(4,18.5)
    frame1.set_ylim(0.05,40)
    pngname = '../output/AGNI/AGNI_spectra/AGNI_spectrum_' + str(round(coords[i][0],2)) + '_' + str(round(coords[i][1],2)) + '.png'
    fig.savefig(pngname)